# Helpers

In [53]:
from collections import defaultdict
from math import floor, sqrt
from pprint import pprint
from typing import Generator

# Tricks

## 1. Unique Representation of Anagrams (Grouping)

In [54]:
strings: list[str] = ["eat", "tea", "tan", "ate", "nat", "bat"]

### Using List Counts and ASCII Values

In [112]:
def groupgen(strings: list[str]) -> list[list[str]]:
    """Group anagrams."""
    groups: dict[str, list[str]] = defaultdict(list)
    for string in strings:
        counts: list[int] = [0 for _ in range(26)]
        for char in string:
            counts[ord(char) - ord("a")] += 1
        groups["".join(map(str, counts))].append(string)

    return groups

pprint(groupgen(strings))

defaultdict(<class 'list'>,
            {'10000000000001000001000000': ['tan', 'nat'],
             '10001000000000000001000000': ['eat', 'tea', 'ate'],
             '11000000000000000001000000': ['bat']})


### Using k Distinct Primes

In [113]:
def is_prime(num: int) -> bool:
    """Checks whether a number is prime."""
    
    if num % 6 not in {1, 5}:
        return False

    for div in range(2, floor(sqrt(num)) + 1):
        if num % div == 0:
            return False

    return True

def primegen() -> int:
    """Prime number generator."""
    yield from [2, 3]
    num: int = 5
    while True:
        if is_prime(num):
            yield num
        num += 1

# Generate primes
gen: Generator[int, int, int] = primegen()
primes: dict[str, int] = dict(zip("abcdefghijklmnopqrstuvwzyz", [next(gen) for _ in range(26)]))

# Generate groups
groups: dict[int, list[str]] = defaultdict(list)
for string in strings:
    prod: int = 1
    for char in string:
        prod *= primes[char]
    groups[prod].append(string)

pprint(groups)

defaultdict(<class 'list'>,
            {426: ['bat'],
             1562: ['eat', 'tea', 'ate'],
             6106: ['tan', 'nat']})


## 2. MinimumArea Rectangle

In [114]:
points: list[int] = [(1, 1), (1, 3), (3, 1), (3, 3), (2, 2)]

In [115]:
#
#   (1, 3)     (3, 3)
#     *          *
#
#     *          *
#   (1, 1)     (3, 1)
#

def min_rec_area(points: list[tuple[float, float]]) -> float:
    seen: set[tuple[int, int]] = set(map(tuple, points))

    res: float = float("inf")
    for i in range(len(points)):
        p1: tuple[int, int] = points[i]
        # No need to iterate through all points in the second loop
        # as at one point, all of the diagonals will be checked anyway
        for j in range(i):
            p2: tuple[int, int] = points[j]
            # If the points form a diagonal
            if p1[0] != p2[0] and p1[1] != p2[1]:
                # And if we can find another diagonal
                if {(p1[0], p2[1]), (p2[0], p1[1])} <= seen:
                    # Properly update the minimum
                    res = min(res, abs((p1[0] - p2[0]) * (p1[1] - p2[1])))


    # In the end, if `res` has changed, return that value
    # Otherwise, return `0`
    return res if res != float("inf") else 0

pprint(min_rec_area(points))

4


## 3. Balanced Parantheses

The idea here is simple. The problem can be solved by applying the following procedures:

- While there are characters to go through:
    - If we stumble into an opening paranthesis, push it onto the stack
    - If we stumble into a closing paranthesis:
        - Make sure the stack is not empty
        - Make sure that the last item on the stack was the opening one
- Make sure that the stack is empty

In [2]:
string1: str = "((adasdsadl))a{{asldkasjdlsalkasdj}aslkdjaslk}sd(asdad)()a{asdlaksj{{asdkljadskl}}asdlkajdhl}s()sajdkla(()())das()(asda(asdas))(asd[[]asdas[[kjdsf]aslkdjas][]]asd)d(sadasd)"
string2: str = "((adasdsadl))a{{asldkasjdlsalkasdj}aslkdjaslk}sd(asdad)()a{asdlaksj{{asdkljadskl}}asdlkajdhl}s()sajdkla(()())das()(asda(asdas))(asd[[]asdas[[kjdsf]aslkdjas][]]asd)d(sadasd)["

In [3]:
def is_balanced(string: str) -> bool:
    """Checks if a string contains balanced sequence of parantheses."""

    stack: list[str] = []
    closing: dict[str, str] = {")": "(", "}": "{", "]": "["}
    opening: set[str] = set(closing.values())
    for char in string:
        if char in opening:
            stack.append(char)
        elif char in closing and (not stack or closing[char] != stack.pop()):
            return False

    return not stack

print(is_balanced(string1))
print(is_balanced(string2))

True
False
